Step 5 - Fold Change Heatmaps

In this step you will obtain heatmaps for each timepoint of your analysis. The results are expressed as a fold change between studied strain and control in particular time and condition. Results which are significant compared to control have annotated FC values, moreover boxes which have significant difference between modification variant of a specific TF are marked with black outline.

In [32]:
import altair as alt
from datetime import datetime
import vl_convert as vlc
import glob
import pandas as pd
import os

In [33]:
# Import the data

IMPORT_PATH = os.path.join(os.getcwd(), "output_data")

# Use the latest excel file as an input (default option)
available_results = glob.glob(os.path.join(IMPORT_PATH, '*growth_data.xlsx'))
latest_file = max(available_results)
data = pd.read_excel(latest_file)

anova = glob.glob(os.path.join(IMPORT_PATH, '*ANOVA_processing_vs_control.xlsx'))
latest_file_anova = max(anova)
data_anova = pd.read_excel(latest_file_anova)

'''
Or enter the exact filename generated in the 01_data_import.ipynb script as an input

data = pd.read_excel('XXX_growth_data.xlsx')
'''

print(latest_file)
print(latest_file_anova)

\\wsl.localhost\Ubuntu\home\marysia\stress_resistance_msc\output_data\2025-05-13_growth_data.xlsx
\\wsl.localhost\Ubuntu\home\marysia\stress_resistance_msc\output_data\2025-10-21_ANOVA_processing_vs_control.xlsx


In [34]:
data['condition_time'] = data['condition'] + '_' + data['time'].astype(str)

data_copy = data.copy()

control_data = data_copy[data_copy['modification'] == 'control'].reset_index()
modif_data = data_copy[data_copy['modification'] != 'control'].reset_index()

In [35]:
print(data_copy.head())

  ECY_number  absorbance_600       TF modification  growth condition  time  \
0        352            1.00  control      control    1.00      test     2   
1        642            0.40    TF011           OE    0.40      test     2   
2        643            0.42    TF011           OE    0.42      test     2   
3        644            0.44    TF011           OE    0.44      test     2   
4        645            0.60     Mhy1           OE    0.60      test     2   

       strain_name          variant_column condition_time  
0  control_control  control_control_test_2         test_2  
1         TF011_OE         TF011_OE_test_2         test_2  
2         TF011_OE         TF011_OE_test_2         test_2  
3         TF011_OE         TF011_OE_test_2         test_2  
4          Mhy1_OE          Mhy1_OE_test_2         test_2  


In [36]:
merged_data = pd.merge(
    modif_data,
    control_data,
    on='condition_time',
    how='outer',
    suffixes= ('_modif', '_control')
)

merged_data = merged_data.drop(columns=['index_modif', 'absorbance_600_modif', 'index_control', 'absorbance_600_control', 
                                        'ECY_number_control', 'TF_control', 'modification_control', 'condition_control',
                                        'time_control', 'strain_name_control'])

In [38]:
merged_data['fold_change'] = merged_data['growth_modif'] / merged_data['growth_control']
merged_data['relative_change'] = (merged_data['growth_modif'] - merged_data['growth_control']) / merged_data['growth_control']

In [44]:
merged_data_anova = pd.merge(merged_data, data_anova[['variant_column_modif', 'p-adj']], on='variant_column_modif', how='outer')

In [45]:
print(merged_data_anova.head())

  ECY_number_modif TF_modif modification_modif  growth_modif condition_modif  \
0              642    TF011                 OE          0.40            test   
1              642    TF011                 OE          0.40            test   
2              643    TF011                 OE          0.42            test   
3              643    TF011                 OE          0.42            test   
4              644    TF011                 OE          0.44            test   

   time_modif strain_name_modif variant_column_modif condition_time  \
0           2          TF011_OE      TF011_OE_test_2         test_2   
1           2          TF011_OE      TF011_OE_test_2         test_2   
2           2          TF011_OE      TF011_OE_test_2         test_2   
3           2          TF011_OE      TF011_OE_test_2         test_2   
4           2          TF011_OE      TF011_OE_test_2         test_2   

   growth_control  variant_column_control  fold_change  relative_change  p-adj  
0          

In [ ]:
VISUALISATION_PATH = os.path.join(os.getcwd(), "visualisations")

current_date = datetime.now().strftime("%Y-%m-%d")

In [54]:
timepoints = merged_data_anova['time_modif'].unique()

n_x = merged_data_anova['condition_modif'].nunique()
n_y = merged_data_anova['strain_name_modif'].nunique()

box_size = 30
chart_width = n_x * box_size
chart_height = n_y * box_size

min_value = merged_data_anova['relative_change'].min()
max_value = merged_data_anova['relative_change'].max()

for time in timepoints:
    one_time = merged_data_anova[merged_data_anova['time_modif'] == time]

    base = alt.Chart(one_time).encode(
        x=alt.X('condition_modif:N', title='Condition'),
        y=alt.Y('strain_name_modif:N', title='Strain')
    )

    # Heatmap (mean over duplicates)
    heatmap = base.mark_rect().encode(
        color=alt.Color(
            'mean(relative_change):Q',
            title='Relative Change',
            scale=alt.Scale(
                domain=[min_value, 0, max_value],
                range=['#122ee3', 'white', '#e31d12'],
                interpolate='rgb'
            )
        )
    )

    
    annotation = (
        base
        .transform_aggregate(
            mean_rc='mean(relative_change)',
            p_adj='mean(p-adj)',  # or 'mean(p-adj)' depending on your stats
            groupby=['condition_modif', 'strain_name_modif']
        )
        .transform_filter(alt.datum.p_adj < 0.05)  # use the aggregated name p_adj
        .mark_text(
            fontSize=10,
            color='black'
        )
        .encode(
            text=alt.Text('mean_rc:Q', format='.1f')  # label with the cell’s mean
        )
    )
    
    # 🔹 Combine both layers
    chart = (heatmap + annotation).properties(
        title=f'Relative Change Heatmap - Time: {time}',
        width=chart_width,
        height=chart_height
    )
    
    chart.display()

    # 🔹 Save chart
    output_filename = f"{current_date}_heatmap_time_{time}.png"
    output_path = os.path.join(VISUALISATION_PATH, output_filename)
    chart.save(output_path, scale_factor=2)  # use scale_factor instead of ppi


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)